# Extract airport and seaport codes from uktradeinfo.com Port_codes spreadsheet

> The air and seaport codes are for use when completing Customs documentation for UK trade with countries outside the European Union.

> The codes are to be used to complete the Location of Goods box (Box 30 of the SAD).

> There are two types of airports - Designated Airports and Non-designated. Non-designated Airports are those with a Certificate of Agreement (CoA) or a military base operating under a Memorandum of Understanding (MoU) or Agreement (MoA).

> Seaports in upper case are trade ports for which trade statistics are available. Ports in lower case are not identified separately for trade statistics purposes, but their trade is included in the statistics for the ports shown in upper case.


Load the Excel spreadsheet and store a copy locally

In [1]:
from databaker.framework import *
import requests
import json
from pathlib import Path
from collections import defaultdict
from string import capwords

ports_file = Path('in/Port_codes.xls')
if not (ports_file.exists() and ports_file.is_file()):
    response = requests.get('https://www.uktradeinfo.com/CodesAndGuides/Documents/Port_codes.xls')
    with open(ports_file, 'wb') as f:
        f.write(response.content)

sheets = loadxlstabs(ports_file)


Loading in/Port_codes.xls which has size 110080 bytes
Table names: ['Information', 'Airport codes', 'Seaport codes', 'Other codes']


The first tab/sheet is a description and the second tab should be named 'Airport codes'.

The 'alpha codes' are the International Air Transport Association (IATA) airport codes, which we'll use as the identity for each airport. We'll collect the rest of the information for each airport as is and output a JSON representation for further processing.

In [2]:
assert len(sheets) == 4
airport_codes = sheets[1]
assert airport_codes.name == 'Airport codes'

airports = {}
for airport in airport_codes.excel_ref('A1').fill(DOWN):
    airports[airport.shift(RIGHT).value] = {
        'label': airport.value,
        'alpha': airport.shift(RIGHT).value,
        'seq': airport.shift(RIGHT).shift(RIGHT).value,
        'type': airport.shift(RIGHT).shift(RIGHT).shift(RIGHT).value
    }
with open('out/airports.json', 'w') as airport_file:
    json.dump(airports, airport_file, indent=2)

print(json.dumps(airports, indent=2))


{
  "ABZ": {
    "label": "Aberdeen Airport",
    "alpha": "ABZ",
    "seq": "449",
    "type": "Designated"
  },
  "AOA": {
    "label": "All Other Airports",
    "alpha": "AOA",
    "seq": "481",
    "type": "Not applicable"
  },
  "BFS": {
    "label": "Belfast International Airport",
    "alpha": "BFS",
    "seq": "427",
    "type": "Designated"
  },
  "BQH": {
    "label": "Biggin Hill Airport",
    "alpha": "BQH",
    "seq": "474",
    "type": "Designated"
  },
  "BHX": {
    "label": "Birmingham Airport",
    "alpha": "BHX",
    "seq": "433",
    "type": "Designated"
  },
  "BLK": {
    "label": "Blackpool International Airport",
    "alpha": "BLK",
    "seq": "459",
    "type": "Designated"
  },
  "BOH": {
    "label": "Bournemouth (Hurn) Airport",
    "alpha": "BOH",
    "seq": "441",
    "type": "Designated"
  },
  "BRS": {
    "label": "Bristol Airport",
    "alpha": "BRS",
    "seq": "438",
    "type": "Designated"
  },
  "BZN": {
    "label": "Brize Norton",
    "alpha": "

The next tab is for seaports. There's more information codified in this table, e.g. a label all in uppercase means that this is the canonical name for the various ports that the statistics will be grouped together into.

The JSON output captures this, using the alpha code as the primary ID and each record can then list the labels of the ports that are included in this group, if any.

In [3]:

seaport_codes = sheets[2]
assert seaport_codes.name == 'Seaport codes'

seaports = defaultdict(dict)
for seaport in seaport_codes.excel_ref('A1').fill(DOWN):
    label = seaport.value
    alpha = seaport.shift(RIGHT).value
    seq = seaport.shift(RIGHT).shift(RIGHT).value
    location = seaport.shift(RIGHT).shift(RIGHT).shift(RIGHT).value
    comment = seaport.shift(RIGHT).shift(RIGHT).shift(RIGHT).shift(RIGHT).value
    
    if 'seq' in seaports[alpha]:
        assert seaports[alpha]['seq'] == seq
    else:
        seaports[alpha]['seq'] = seq
    
    if location != 'Not applicable':
        if 'location' in seaports[alpha]:
            assert seaports[alpha]['location'] == location
        else:
            seaports[alpha]['location'] = location
    
    if comment != "":
        if 'comment' in seaports[alpha]:
            assert seaports[alpha]['comment'] == comment
        else:
            seaports[alpha]['comment'] = comment

    if label.isupper():
        seaports[alpha]['label'] = capwords(label)
    else:
        if 'includes' in seaports[alpha]:
            seaports[alpha]['includes'].append(label)
        else:
            seaports[alpha]['includes'] = [label]

with open('out/seaports.json', 'w') as seaport_file:
    json.dump(seaports, seaport_file, indent=2)

print(json.dumps(seaports, indent=2))

{
  "ABD": {
    "seq": "164",
    "location": "Scotland",
    "label": "Aberdeen"
  },
  "TRT": {
    "seq": "203",
    "location": "England & Wales",
    "includes": [
      "Althorpe",
      "Beckingham",
      "Burton Stather",
      "Flixborough Wharf",
      "Gainsborough",
      "Grove Wharves",
      "Guness Wharf",
      "Keadby",
      "Neap House",
      "Point BID",
      "Scunthorpe"
    ],
    "label": "Trent"
  },
  "BLY": {
    "seq": "176",
    "location": "England & Wales",
    "includes": [
      "Amble",
      "Berwick"
    ],
    "label": "Blyth"
  },
  "GRK": {
    "seq": "154",
    "location": "Scotland",
    "includes": [
      "Annan",
      "Barcaldine",
      "Campbeltown",
      "Faslane",
      "Finnart",
      "Furnace",
      "Garlieston",
      "Hunterston",
      "Islay",
      "Oban",
      "Palnackie",
      "Stranraer"
    ],
    "label": "Greenock"
  },
  "WAT": {
    "seq": "076",
    "location": "England & Wales",
    "includes": [
      "Appledor